# YCrCb image compression

Removing the redundancy in the color domain with the YCrCb transform.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
!ln -sf ~/quantization/deadzone_quantizer.py .
!ln -sf ~/quantization/midtread_quantizer.py .
!ln -sf ~/quantization/midrise_quantizer.py .
!ln -sf ~/quantization/distortion.py .
!ln -sf ~/quantization/information.py .
!ln -sf ~/MRVC/src/YCrCb.py .
!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/image_3.py .
!ln -sf ~/MRVC/src/image_1.py .
!ln -sf ../common.py .
import deadzone_quantizer as deadzone
import midtread_quantizer as midtread
import midrise_quantizer as midrise
import YCrCb
import distortion
import information
import image_3 as RGB_image
import image_1 as gray_image
import colored
import common

## Configuration

In [ ]:
# Prefix of the RGB image to be quantized.

home = os.environ["HOME"]
#fn = home + "/MRVC/sequences/stockholm/"
fn = home + "/MRVC/sequences/lena_color/"

YCrCb_components = ['Y', 'Cr', 'Cb']

# Number of quantization steps.
N_Q_steps = 8
#Q_steps = [256*i/N_Q_steps for i in range(N_Q_steps + 1, 0, -1)]
Q_steps = [2**i for i in range(7, -1, -1)]
print(Q_steps)

#quantizer = midtread
quantizer = deadzone
#quantizer = midrise

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(np.uint16)
common.show(RGB_img, fn + "000.png")

In [ ]:
RGB_img.shape

In [ ]:
RGB_img.dtype

## (RGB -> YCrCb) transform of the image

In [ ]:
YCrCb_img = YCrCb.from_RGB(RGB_img)
print(YCrCb_img.dtype)
Y_img = YCrCb_img[...,0]
Cr_img = YCrCb_img[...,1]
Cb_img = YCrCb_img[...,2]

In [ ]:
common.show_gray(Y_img, fn + "000 (Y comp.)")

In [ ]:
common.show_gray(Cr_img, fn + "000 (Cr comp.)")

In [ ]:
common.show_gray(Cb_img, fn + "000 (Cb comp.)")

The YCbCr components ranges between [0, 255].

## Energy of the YCrCb components

In [ ]:
Y_avg_energy = information.average_energy(Y_img)
Cr_avg_energy = information.average_energy(Cr_img)
Cb_avg_energy = information.average_energy(Cb_img)
print(f"Average energy in the Y image = {Y_avg_energy}")
print(f"Average energy in the Cr image = {Cr_avg_energy}")
print(f"Average energy in the Cb image = {Cb_avg_energy}")
total_YCrCb_avg_energy = Y_avg_energy + Cr_avg_energy + Cb_avg_energy
print(f"Total average energy (computed by adding the energies of the YCrCb components) = {total_YCrCb_avg_energy}")
print(f"Total RGB average energy (computed directly from the RGB image) = {information.average_energy(RGB_img)*3}")

The forward YCrCb transform is not energy preserving, but is close.

In [ ]:
RGB_recons_img = YCrCb.to_RGB(YCrCb_img)
print(f"Total RGB average energy of reconstruction () = {information.average_energy(RGB_recons_img)*3}")

Neither the backward (inverse) YCrCb transform is. However, it could be an error generated by the use of floating point arithmetic.

## (RGB <-> YCrCb) transform error

In [ ]:
common.show(RGB_recons_img, fn + "000.png (YCrCb recons)")

In [ ]:
np.array_equal(RGB_img, RGB_recons_img)

In [ ]:
print(RGB_img.max(), RGB_img.min())

In [ ]:
print(RGB_recons_img.max(), RGB_recons_img.min())

In [ ]:
common.show(RGB_img-RGB_recons_img, "Reconstruction error (rounding error)")

The YCrCb transform is irreversible. In general, only integer arithmetic operations guarantees reversibility.

## Synthesis filters gains

The synthesis filters gains are important because the quantization steps of each YCrCb component should be adjusted in order to effectively provide the desired number of [bins](http://www.winlab.rutgers.edu/~crose/322_html/quantization.pdf) (different dequantized values) in each component.

In [ ]:
def print_info(val):
    Y_delta = np.array([val, 0, 0]).astype(np.uint8).reshape(1,1,3)
    RGB_Y_delta = YCrCb.to_RGB(Y_delta)
    RGB_energy_Y_delta = information.energy(RGB_Y_delta)
    
    Cr_delta = np.array([0, val, 0]).astype(np.uint8).reshape(1,1,3)
    RGB_Cr_delta = YCrCb.to_RGB(Cr_delta)
    RGB_energy_Cr_delta = information.energy(RGB_Cr_delta)
    
    Cb_delta = np.array([0, 0, val]).astype(np.uint8).reshape(1,1,3)
    RGB_Cb_delta = YCrCb.to_RGB(Cb_delta)
    RGB_energy_Cb_delta = information.energy(RGB_Cb_delta)
    
    zero = np.array([0, 0, 0]).astype(np.uint8).reshape(1,1,3)
    RGB_zero = YCrCb.to_RGB(zero)
    RGB_energy_zero = information.energy(RGB_zero)

    print(f"Energy of {Y_delta} in the RGB domain ({RGB_Y_delta}) = {RGB_energy_Y_delta}")
    print(f"Energy of {Cr_delta} in the RGB domain ({RGB_Cr_delta}) = {RGB_energy_Cr_delta}")
    print(f"Energy of {Cb_delta} in the RGB domain ({RGB_Cb_delta}) = {RGB_energy_Cb_delta}")
    print(f"Energy of {zero} in the RGB domain ({RGB_zero}) = {RGB_energy_zero}")
    
    max_ = max(RGB_energy_Y_delta, RGB_energy_Cr_delta, RGB_energy_Cb_delta)
    Y_relative_gain = RGB_energy_Y_delta / max_
    Cr_relative_gain = RGB_energy_Cr_delta / max_
    Cb_relative_gain = RGB_energy_Cb_delta / max_
    print(f"Relative gain of Y component = {Y_relative_gain}")
    print(f"Relative gain of Cr component = {Cr_relative_gain}")
    print(f"Relative gain of Cb component = {Cb_relative_gain}")
    
print_info(255)
print()
print_info(1)
print()
print_info(0)

Unfortunately, as it can be seen, the gain of the filters of the inverse transform depends on the value of the "delta" in the YCrCb domain. This is a consequence of the no orthogonality of the transform (the analysis filters are not independent, and the same happens with the synthesis filters). Therefore, if we minimize the quantization error in the YCrCb domain we will not minimize the quantization error in the RGB domain. This also implies that the distortion cannot be measured in the YCrCb domain when we are performing a Rate/Distortion Optimization (RDO).

## Noise in the YCrCb domain

To decide how to quantize, it is necessary to known how the amplitudes the electronic noise of the input image is transformed to the DCT domain. To see this, we will generate a random image with ([normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) with mean 0, DCT-it, and check where the transformed noise has its mean. Notice that, by definition, the noise cannot be decorrelated by transforms, and therefore, the noise is simply transfered to the transform domain.

In [ ]:
# loc = mean, scale=standard deviation, size=number of samples
RGB_noise = np.random.normal(loc=0, scale=10, size=512*512*3).reshape(512,512,3).astype(np.uint16)

In [ ]:
common.show(RGB_noise, "Gaussign RGB noise")

In [ ]:
YCrCb_noise = YCrCb.from_RGB(RGB_noise)

In [ ]:
common.show_gray(YCrCb_noise[..., 0], "Gaussign Y noise")

In [ ]:
common.show_gray(YCrCb_noise[..., 1], "Gaussign Cr noise")

In [ ]:
common.show_gray(YCrCb_noise[..., 2], "Gaussign Cr noise")

The Y component keeps the noise in the low aplitudes. However, the Cr and Cb components add 128. A dead-zone quantizer can manage such shifts, because it is almost symmetric. However, in the case of using a companded quantizer, the "high frequency" components should be 128 substracted before using the quantizer.

## Simple quantization in the YCrCb domain ($\Delta_{\text{Y}} = \Delta_{\text{Cb}} = \Delta_{\text{Cr}}$)
Notice that the distortion must be measured in the RGB domain.

In [ ]:
def YCrCb_same_delta_RD_curve(RGB_img, Q_steps, quantizer):
    YCrCb_img = YCrCb.from_RGB(RGB_img).astype(np.int16)
    points = []
    YCrCb_img -= 128
    for Q_step in Q_steps:
        YCrCb_y, k = quantizer.quan_dequan(YCrCb_img, Q_step)
        k_min = np.min(k)
        #rate = common.bits_per_color_pixel((k - k_min).astype(np.uint8), str(Q_step) + '_')
        rate = common.bits_per_color_pixel(k.astype(np.uint8), str(Q_step) + '_')
        #RGB_y = YCrCb.to_RGB(YCrCb_y)
        RGB_y = YCrCb.to_RGB(YCrCb_y + 128)
        _distortion = distortion.MSE(RGB_img, RGB_y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

def YCrCb_same_delta_RD_curve2(RGB_img, Q_steps, quantizer):
    YCrCb_img = YCrCb.from_RGB(RGB_img).astype(np.int16)
    points = []
    #YCrCb_img -= 128
    for Q_step in Q_steps:
        YCrCb_y, k = quantizer.quan_dequan(YCrCb_img, Q_step)
        k_min = np.min(k)
        #rate = common.bits_per_color_pixel((k - k_min).astype(np.uint8), str(Q_step) + '_')
        rate = common.bits_per_color_pixel(k.astype(np.uint8), str(Q_step) + '_')
        RGB_y = YCrCb.to_RGB(YCrCb_y)
        #RGB_y = YCrCb.to_RGB(YCrCb_y + 128)
        _distortion = distortion.MSE(RGB_img, RGB_y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

def YCrCb_same_delta_RD_curve3(RGB_img, Q_steps, quantizer):
    YCrCb_img = YCrCb.from_RGB(RGB_img).astype(np.int16)
    points = []
    Y_min = 128 #np.min(YCrCb_img[..., 0])
    Cr_min = 128 #np.min(YCrCb_img[..., 1])
    Cb_min = 128 #np.min(YCrCb_img[..., 2])
    print(Y_min, Cr_min, Cb_min)
    YCrCb_img[..., 0] -= Y_min
    YCrCb_img[..., 1] -= Cr_min
    YCrCb_img[..., 2] -= Cb_min
    for Q_step in Q_steps:
        YCrCb_y, k = quantizer.quan_dequan(YCrCb_img, Q_step)
        k_min = np.min(k)
        rate = common.bits_per_color_pixel((k - k_min).astype(np.uint8), str(Q_step) + '_')
        #rate = common.bits_per_color_pixel(k.astype(np.uint8), str(Q_step) + '_')
        #RGB_y = YCrCb.to_RGB(YCrCb_y + 128)
        YCrCb_y[..., 0] += Y_min
        YCrCb_y[..., 1] += Cr_min
        YCrCb_y[..., 2] += Cb_min
        RGB_y = YCrCb.to_RGB(YCrCb_y)
        _distortion = distortion.MSE(RGB_img, RGB_y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

YCrCb_same_delta_RD_points = YCrCb_same_delta_RD_curve(RGB_img, Q_steps, quantizer)
YCrCb_same_delta_RD_points2 = YCrCb_same_delta_RD_curve2(RGB_img, Q_steps, quantizer)
YCrCb_same_delta_RD_points3 = YCrCb_same_delta_RD_curve3(RGB_img, Q_steps, quantizer)

## Let's see the RD curve

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YCrCb_same_delta_RD_points), c='g', marker='.', label="$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Cr}} = \Delta_{\mathrm{Cb}}$", linestyle="dashed")
pylab.plot(*zip(*YCrCb_same_delta_RD_points2), c='b', marker='.', label="$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Cr}} = \Delta_{\mathrm{Cb}}$", linestyle="dashed")
pylab.plot(*zip(*YCrCb_same_delta_RD_points3), c='r', marker='.', label="$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Cr}} = \Delta_{\mathrm{Cb}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

## Optimal RD curve

The method `YCrCb_same_delta_RD_curve(img, Q_steps, quantizer)` generates a RD curve where each point is the result of using $\Delta_{\text{Y}} = \Delta_{\text{Cb}} = \Delta_{\text{Cr}}$. However, a better (at least with more points) RD curve can be generated with:

1. Convert the image from RGB to YCrCb.
2. The RD curve of each YCrCb channel is computed, for a number of quantization steps, measuring the distortion in the RGB domain (remember that the YCrCb transform is not orthogonal and therefore, the distortion in the RGB domain cannot be estimated in the YCrCb domain).  We will quantize one component at each iteration and the rest of components will be unquantized. This is necessary to ensure that the low-pass component (Y) is always considered in each reconstruction. Otherwise, the distortion (at least using the MSE) will not be estimated correctly.
3. Compute the slope of each segment of the RD curve. Except for the most left point, the slopes are computed as the average between the slopes of the straight lines that connect to the corresponding point.
4. For each quantization step, sort the RD points by their slope.
5. Recompute the optimal RD curve using the quantization steps provided by the sorted RD points.

## RD curve of each YCrCb component

In [ ]:
def YCrCb_RD_curve_per_component(RGB_img, Q_steps, Q, components):
    YCrCb_img = YCrCb.from_RGB(RGB_img).astype(np.int16)
    YCrCb_img -= 128
    YCrCb_img_copy = YCrCb_img.copy()
    N_components = len(components)
    RD_points = []
    for c in range(N_components):
        RD_points.append([])
    for Q_step in Q_steps:
        YCrCb_k = Q.quantize(YCrCb_img, Q_step)
        for component_index in range(N_components):
            component_name = components[component_index]
            YCrCb_y = YCrCb_img_copy.copy()
            YCrCb_y[..., component_index] = Q.dequantize(YCrCb_k[..., component_index], Q_step)
            rate = common.bits_per_gray_pixel(YCrCb_k[..., component_index] + 128, str(Q_step) + '_' + component_name)
            RGB_y = YCrCb.to_RGB(YCrCb_y + 128)
            _distortion = distortion.MSE(RGB_img, RGB_y)
            #common.show(RGB_y, components[component_index] + ' ' + str(Q_step))
            RD_points[component_index].append((rate, _distortion, component_name, Q_step))
            print(f"{components[component_index]} q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return RD_points

YCrCb_curve_per_component = YCrCb_RD_curve_per_component(RGB_img, Q_steps, quantizer, YCrCb_components)
YCrCb_curve_per_component

## Display the curves of the components

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*[(i[0], i[1]) for i in YCrCb_curve_per_component[0]]), c='r', marker='.', label="$\mathrm{Y}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in YCrCb_curve_per_component[1]]), c='g', marker='.', label="$\mathrm{Cr}}$", linestyle="dashed")
pylab.plot(*zip(*[(i[0], i[1]) for i in YCrCb_curve_per_component[2]]), c='b', marker='.', label="$\mathrm{Cb}}$", linestyle="dashed")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE in the RGB Domain")
pylab.legend(loc='upper right')
pylab.show()

## Compute the slope of each point

In [ ]:
Y_slopes = common.compute_slopes(YCrCb_curve_per_component[0])
Cr_slopes = common.compute_slopes(YCrCb_curve_per_component[1])
Cb_slopes = common.compute_slopes(YCrCb_curve_per_component[2])

In [ ]:
Y_slopes

In [ ]:
Cr_slopes

In [ ]:
Cb_slopes

## Filter the curves
Remove those RD points that do not belong to the convex-hull

In [ ]:
Y_slopes = common.filter_slopes(Y_slopes)
Y_slopes

In [ ]:
Cr_slopes = common.filter_slopes(Cr_slopes)
Cr_slopes

In [ ]:
Cb_slopes = common.filter_slopes(Cb_slopes)
Cb_slopes

## Sort the slopes at each quantization step
The TPs generated in a component must be used in order.

In [ ]:
_i = 0
sorted_slopes = []
#print(Q_steps)
for _Q_step in range(len(Q_steps)):
    #print(_i)
    YCrCb_slopes_for_such_Q_step = [Y_slopes[_i], Cr_slopes[_i], Cb_slopes[_i]]
    sorted_slopes.append(sorted(YCrCb_slopes_for_such_Q_step, key=lambda x: x[0])[::-1])
    #sorted_slopes.append(YCrCb_slopes_for_such_Q_step)
    #print(YCrCb_slopes_for_such_Q_step)
    _i += 1

#all_slopes = Y_slopes + Cr_slopes + Cb_slopes
#sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

In [ ]:
one_list = []
for _list in sorted_slopes:
    one_list += _list
    print(_list)
one_list
sorted_slopes = one_list.copy()

In [ ]:
all_slopes = Y_slopes + Cr_slopes + Cb_slopes
sorted_slopes = sorted(all_slopes, key=lambda x: x[0])[::-1]
sorted_slopes

Crominance without luminance makes not sense from a visual point of view, but these are the number :-/

In [ ]:
#sorted_slopes[0], sorted_slopes[2] = sorted_slopes[2], sorted_slopes[0]
#sorted_slopes

And finally, let's compute the RD curve (remember that the previous points are only an estimation of the order in which the quantization steps should be increased in each component to build the RD curve, not the real RD curve that measures the distortion in the RGB domain).

In [ ]:
def YCrCb_optimal_curve_(RGB_img, sorted_slopes, quantizer, components):
    points = []
    Q_steps_per_component = [256, 256, 256]
    YCrCb_img = YCrCb.from_RGB(RGB_img).astype(np.int16)
    k = np.empty_like(YCrCb_img)
    YCrCb_y = np.empty_like(YCrCb_img)
    YCrCb_img -= 128
    for i in sorted_slopes:
        print(i)
        #point = i[1]
        #component = point[2]
        #Q_step = point[3]
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            YCrCb_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(YCrCb_img[..., component_index], Q_step)
        k = k.astype(np.uint8)
        rate = common.bits_per_color_pixel(k, str(Q_steps_per_component) + '_')
        y_RGB = YCrCb.to_RGB(YCrCb_y + 128)
        #y_RGB = YCrCb.to_RGB(y)
        _distortion = distortion.MSE(RGB_img, y_RGB)
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

def YCrCb_optimal_curve(RGB_img, sorted_slopes, quantizer, components):
    points = []
    Q_steps_per_component = [256, 256, 256]
    YCrCb_img = YCrCb.from_RGB(RGB_img).astype(np.int16)
    k = np.empty_like(YCrCb_img)
    YCrCb_y = np.empty_like(YCrCb_img)
    YCrCb_img -= 128
    for i in sorted_slopes:
        print(i)
        component, Q_step = i[1], i[2]
        Q_steps_per_component[components.index(component)] = Q_step
        for c, Q_step in zip(components, Q_steps_per_component):
            component_index = components.index(c)
            YCrCb_y[..., component_index], k[..., component_index] = quantizer.quan_dequan(YCrCb_img[..., component_index], Q_step)
        k_min = np.min(k[..., component_index])
        #k = (k - k_min).astype(np.uint8)
        k = k.astype(np.uint8)
        rate = common.bits_per_color_pixel(k, str(Q_steps_per_component) + '_')
        #RGB_y = YCrCb.to_RGB(YCrCb_y)
        RGB_y = YCrCb.to_RGB(YCrCb_y + 128)
        _distortion = distortion.MSE(RGB_img, RGB_y)
        #common.show(y_RGB, f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
        points.append((rate, _distortion))
        print(f"Q_step={Q_steps_per_component}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

YCrCb_optimal_RD_points = YCrCb_optimal_curve(RGB_img, sorted_slopes, quantizer, YCrCb_components)
YCrCb_optimal_RD_points

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*YCrCb_same_delta_RD_points), c='b', marker='o', label="$\Delta_{\mathrm{Y}} = \Delta_{\mathrm{Cb}} = \Delta_{\mathrm{Cr}}$", linestyle="dashed")
pylab.plot(*zip(*YCrCb_optimal_RD_points), c='r', marker='x', label="Optimal", linestyle="dotted")
pylab.title("Rate/Distortion")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("MSE in RGB domain")
pylab.legend(loc='upper right')
pylab.show()

## Conclusion

In general, the basic quantization scheme that uses the same quantization step size for all the components performs almost optimal.